## Homework 7

In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import math
import scipy.stats as stats
import seaborn as sns
import statsmodels.api as sm
from statsmodels.regression.rolling import RollingOLS
from sklearn.linear_model import LinearRegression

In [2]:
dataname = '../data/gmo_analysis_data.xlsx'
signals = pd.read_excel(dataname,sheet_name = 'signals')
returns = pd.read_excel(dataname,sheet_name = 'returns (total)')
rf = pd.read_excel(dataname,sheet_name = 'risk-free rate')
signals = signals.rename(columns={'Unnamed: 0':'date'}).set_index('date')
returns = returns.rename(columns={'Unnamed: 0':'date'}).set_index('date')
rf = rf.rename(columns={'Unnamed: 0':'date'}).set_index('date')
exe = returns.dropna().subtract(rf['US3M'],axis=0).dropna()
exe

,SPY,GMWAX
date,,
1996-11-30,0.068729,0.040906
1996-12-31,-0.028150,-0.015631
1997-01-31,0.057494,0.010443
1997-02-28,0.005216,0.017915
1997-03-31,-0.048597,-0.019610
...,...,...
2022-06-30,-0.083903,-0.065589
2022-07-31,0.090079,0.032416
2022-08-31,-0.043269,-0.028286


In [3]:
exe11 = exe[:'2011']
exe22 = exe['2012':]

### Part 2 - Analyzing GMO

#### Question 1 and Question 2

In [4]:
def simplemetrics(mid, freq=12):
    metrics = pd.DataFrame(index=mid.columns)
    metrics['mean'] = mid.mean() * freq
    metrics['vol'] = mid.std() * np.sqrt(freq)
    metrics['sharpe'] = metrics['mean'] / metrics['vol']
#     metrics['skewness'] = mid.skew()
#     metrics['kustosis'] = mid.kurtosis() 
    metrics['VaR'] = mid.quantile(.05) 
#     metrics['CVaR'] = mid[mid < metrics['VaR']].mean()
    cummulative_returns = (1 + mid).cumprod()
    rolling_max = cummulative_returns.cummax()
    drawdown = (cummulative_returns - rolling_max)/rolling_max
    metrics['max drawdown'] = drawdown.min()
#     metrics['max'] = mid.max()
    metrics['min'] = mid.min()
    return metrics

In [5]:
simplemetrics(exe11)

,mean,vol,sharpe,VaR,max drawdown,min
SPY,0.039987,0.164976,0.242383,-0.080224,-0.560012,-0.165570
GMWAX,0.015827,0.125011,0.126603,-0.059806,-0.472946,-0.149179


In [6]:
simplemetrics(exe22)

,mean,vol,sharpe,VaR,max drawdown,min
SPY,0.126509,0.143068,0.884254,-0.068658,-0.248125,-0.124734
GMWAX,0.036635,0.091996,0.398229,-0.039686,-0.226046,-0.118650


In [7]:
simplemetrics(exe)

,mean,vol,sharpe,VaR,max drawdown,min
SPY,0.076038,0.156469,0.485962,-0.080006,-0.560012,-0.165570
GMWAX,0.024497,0.112314,0.218112,-0.048293,-0.472946,-0.149179


Q2 a)

Q2 b)

#### Question 3

In [8]:
y = exe11['GMWAX']
x = sm.add_constant(exe11['SPY'])
reg = sm.OLS(y,x, missing = 'drop').fit()
params_df = reg.params.to_frame('Regression Results')
params_df = params_df.rename(index={'const':'alpha'})
params_df.loc[r'$R^{2}$'] = reg.rsquared
params_df

,Regression Results
alpha,-0.000479
SPY,0.539616
$R^{2}$,0.507129


In [9]:
y = exe22['GMWAX']
x = sm.add_constant(exe22['SPY'])
reg = sm.OLS(y,x, missing = 'drop').fit()
params_df = reg.params.to_frame('Regression Results')
params_df = params_df.rename(index={'const':'alpha'})
params_df.loc[r'$R^{2}$'] = reg.rsquared
params_df

,Regression Results
alpha,-0.002874
SPY,0.562232
$R^{2}$,0.764506


In [10]:
y = exe['GMWAX']
x = sm.add_constant(exe['SPY'])
reg = sm.OLS(y,x, missing = 'drop').fit()
params_df = reg.params.to_frame('Regression Results')
params_df = params_df.rename(index={'const':'alpha'})
params_df.loc[r'$R^{2}$'] = reg.rsquared
params_df

,Regression Results
alpha,-0.001416
SPY,0.545600
$R^{2}$,0.577744


a)

b)

### Part  - Forecast Regressions

#### Question 1

In [11]:
shift = signals.shift()
shift['SPY'] = returns['SPY']
shift

,DP,EP,US10Y,SPY
date,,,,
1993-02-28,NaN,NaN,NaN,0.010667
1993-03-31,2.82,4.44,6.03,0.022408
1993-04-30,2.77,4.41,6.03,-0.025589
1993-05-31,2.82,4.44,6.05,0.026970
1993-06-30,2.81,4.38,6.16,0.003667
...,...,...,...,...
2022-06-30,1.57,4.79,2.85,-0.082469
2022-07-31,1.64,5.23,2.98,0.092087
2022-08-31,1.65,4.91,2.67,-0.040802


In [12]:
y = shift['SPY']
x = sm.add_constant(shift['DP'])
reg = sm.OLS(y,x, missing = 'drop').fit()
params_df = reg.params.to_frame('Regression Results')
params_df = params_df.rename(index={'const':'alpha'})
params_df.loc[r'$R^{2}$'] = reg.rsquared
w_DP = 100 * (params_df.loc['alpha'][0]/12 + params_df.loc['DP'][0] * shift['DP'])
r_DP = (w_DP * shift['SPY']).dropna()
params_df

,Regression Results
alpha,-0.009405
DP,0.009430
$R^{2}$,0.009355


In [13]:
y = shift['SPY']
x = sm.add_constant(shift['EP'])
reg = sm.OLS(y,x, missing = 'drop').fit()
params_df = reg.params.to_frame('Regression Results')
params_df = params_df.rename(index={'const':'alpha'})
params_df.loc[r'$R^{2}$'] = reg.rsquared
w_EP = 100 * (params_df.loc['alpha'][0]/12 + params_df.loc['EP'][0] * shift['EP'])
r_EP = (w_EP * shift['SPY']).dropna()
params_df

,Regression Results
alpha,-0.005934
EP,0.003196
$R^{2}$,0.008613


In [14]:
y = shift['SPY']
x = sm.add_constant(shift[['EP','DP','US10Y']])
reg = sm.OLS(y,x, missing = 'drop').fit()
params_df = reg.params.to_frame('Regression Results')
params_df = params_df.rename(index={'const':'alpha'})
params_df.loc[r'$R^{2}$'] = reg.rsquared
w_3f = 100 * (params_df.loc['alpha'][0]/12 + params_df.loc['EP'][0] * shift['EP']\
                                           + params_df.loc['DP'][0] * shift['DP']\
                                           + params_df.loc['US10Y'][0] * shift['US10Y'])
r_3f = (w_3f * shift['SPY']).dropna()
params_df

,Regression Results
alpha,-0.014933
EP,0.002651
DP,0.007979
US10Y,-0.000968
$R^{2}$,0.016333


#### Question 2

In [15]:
def metricsandreg(mid, sec,freq=12):
    metrics = pd.DataFrame(index=['Summary Stats'])
    metrics['mean'] = mid.mean() * freq
    metrics['vol'] = mid.std() * np.sqrt(freq)
    metrics['sharpe'] = metrics['mean'] / metrics['vol']
    cummulative_returns = (1 + mid).cumprod()
    rolling_max = cummulative_returns.cummax()
    drawdown = (cummulative_returns - rolling_max)/rolling_max
    metrics['max drawdown'] = drawdown.min()
    
    y=mid
    x=sm.add_constant(sec.loc[mid.index])
    reg = sm.OLS(y,x,missing = 'drop').fit().params
    metrics[r'$\alpha$'] = reg[0] * freq
    metrics[r'$\beta$'] = reg[1]
    return metrics

In [16]:
metricsandreg(r_DP,shift[['SPY']])

,mean,vol,sharpe,max drawdown,$\alpha$,$\beta$
Summary Stats,0.198349,0.268307,0.739262,-0.833442,0.02067,1.723167


In [17]:
metricsandreg(r_EP,shift[['SPY']])

,mean,vol,sharpe,max drawdown,$\alpha$,$\beta$
Summary Stats,0.163885,0.202401,0.809703,-0.50917,0.032247,1.276654


In [18]:
metricsandreg(r_3f,shift[['SPY']])

,mean,vol,sharpe,max drawdown,$\alpha$,$\beta$
Summary Stats,0.266157,0.332426,0.800648,-0.847623,0.045096,2.14389


#### Question 3

In [19]:
VaR = pd.DataFrame([r_DP.quantile(.05), r_EP.quantile(.05), r_3f.quantile(.05), 
                    shift['SPY'].quantile(.05), 
                    exe['GMWAX'].quantile(.05)],
                   index = ['DP Strat','EP Strat','3-factor Strat','SPY','GMO'], 
                   columns = ['5% VaR'])

VaR

,5% VaR
DP Strat,-0.110054
EP Strat,-0.091809
3-factor Strat,-0.152674
SPY,-0.073936
GMO,-0.048293


#### Question 3B

In [20]:
simplemetrics(r_DP.to_frame('DP Strat').loc['2000':'2011'])

,mean,vol,sharpe,VaR,max drawdown,min
DP Strat,0.054988,0.311868,0.176319,-0.131907,-0.833442,-0.356232


In [21]:
simplemetrics(r_EP.to_frame('DP Strat').loc['2000':'2011'])

,mean,vol,sharpe,VaR,max drawdown,min
DP Strat,0.047148,0.209122,0.225458,-0.098446,-0.50917,-0.191369


In [22]:
simplemetrics(r_3f.to_frame('DP Strat').loc['2000':'2011'])

,mean,vol,sharpe,VaR,max drawdown,min
DP Strat,0.085658,0.357436,0.239647,-0.17863,-0.847623,-0.395808


In [23]:
simplemetrics(rf.loc['2000':'2011'])

,mean,vol,sharpe,VaR,max drawdown,min
US3M,0.023062,0.005785,3.986632,0.000035,0.0,0.000008


#### Question 3C

In [24]:
r_df = r_3f.to_frame('3-factor Strat')
r_df['DP Strat'] = r_DP
r_df['EP Strat'] = r_EP
r_df['rf'] = rf['US3M']

df_riskprem = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_df.columns[:3]:
    df_riskprem[col] = len(r_df[r_df[col] < r_df['rf']])/len(r_df) * 100
    
df_riskprem

,3-factor Strat,DP Strat,EP Strat
% of periods underperforming $r^{f}$,35.674157,35.955056,35.955056


### Part 4 - Out-of-Sample Forecasting

In [25]:
def OOS(df,cl,factors, start):
    y = df[cl]
    x = sm.add_constant(df[factors])
    err,nerr = [],[]
    
    for i,j in enumerate(df.index):
        if i>=start:
            currx = x.iloc[:i]
            curry = y.iloc[:i]
            reg = sm.OLS(curry,currx,missing='drop').fit()
            null_forecast = curry.mean()
            reg_predict = reg.predict(x.iloc[[i]])
            actual = y.iloc[[i]]
            err.append(reg_predict - actual)
            nerr.append(null_forecast - actual)
    RSS = (np.array(err)**2).sum()
    TSS = (np.array(nerr)**2).sum()
    
    return 1 - (RSS/TSS)
OOS(shift,'SPY',['EP'], 60)

-0.00696611303774386

In [26]:
print('EP OOS R-squared is ' + str(round(OOS(shift,'SPY',['EP'], 60),5)))

EP OOS R-squared is -0.00697


In [27]:
def OOS_strat(df,cl,factors, start,weight):
    y = df[cl]
    x = sm.add_constant(df[factors])
    r = []
    
    for i,j in enumerate(df.index):
        if i>=start:
            currx = x.iloc[:i]
            curry = y.iloc[:i]
            reg = sm.OLS(curry,currx,missing='drop').fit()
            pred = reg.predict(x.iloc[[i]])
            w = pred * weight
            r.append((df.iloc[i][cl] * w)[0])
    df_strat = pd.DataFrame(data = r,index = df.iloc[-(len(r)):].index,columns = ['Strat Returns']) 
    return df_strat

In [28]:
OOS_EP = OOS_strat(shift,'SPY',['EP'],60,100)
OOS_EP

,Strat Returns
date,
1998-02-28,0.051217
1998-03-31,0.045644
1998-04-30,0.014350
1998-05-31,-0.022508
1998-06-30,0.035103
...,...
2022-06-30,-0.079378
2022-07-31,0.098652
2022-08-31,-0.040768


In [29]:
metricsandreg(OOS_EP['Strat Returns'],shift[['SPY']])

,mean,vol,sharpe,max drawdown,$\alpha$,$\beta$
Summary Stats,0.081898,0.165352,0.495294,-0.583693,0.035321,0.543527


In [30]:
VaR_oos = pd.DataFrame([OOS_EP['Strat Returns'].quantile(.05), 
                    shift['SPY'].quantile(.05), 
                    exe['GMWAX'].quantile(.05)],
                   index = ['Strt Returns','SPY','GMO'], 
                   columns = ['5% VaR'])

VaR_oos

,5% VaR
Strt Returns,-0.070971
SPY,-0.073936
GMO,-0.048293


In [31]:
metricsandreg(OOS_EP['2000':'2011']['Strat Returns'],shift[['SPY']])

,mean,vol,sharpe,max drawdown,$\alpha$,$\beta$
Summary Stats,0.038768,0.195919,0.197877,-0.583693,0.033332,0.299368


In [32]:
simplemetrics(rf.loc['2000':'2001'])

,mean,vol,sharpe,VaR,max drawdown,min
US3M,0.046787,0.00454,10.305323,0.001517,0.0,0.00145


In [33]:
r_df = OOS_EP.rename(columns={"Strat Returns": "EP Strat"})
r_df['rf'] = rf['US3M']

df_riskprem = pd.DataFrame(data=None, index=[r'% of periods underperforming $r^{f}$'])
for col in r_df.columns[:3]:
    df_riskprem[col] = len(r_df[r_df[col] < r_df['rf']])/len(r_df) * 100
    
df_riskprem

,EP Strat,rf
% of periods underperforming $r^{f}$,38.383838,0.0


D)